## Content
1. [Loading preprocessed data](#1.-Loading-preprocessed-data)  
2. [Solutions](#2.-Solutions)  
    2.1 [Random approach](#2.1-Random-approach)  
    2.2 [String kernels](#2.2-String-kernels)   
      2.2.1 [Spectrum kernel](#2.2.1-Spectrum-kernel)  
      2.2.2 [Presence kernel](#2.2.2-Presence-kernel)  
      2.2.3 [Intersection kernel](#2.2.3-Intersection-kernel)  
    2.3 [Sentence similarity](#2.3-Sentence-similarity)   
3. [Results](#3.-Results)

## 1. Loading preprocessed data

In [1]:
import json
import pandas as pd
from pprint import pprint
from tqdm import tqdm

IN_PATH = '../data/squad/'


def load_data(filename):
    data_frame = pd.read_json(filename)
    data = data_frame.to_dict(orient='list')
    
    return data

In [2]:
train = load_data(IN_PATH + 'train-v1.1-preprocessed.json')
dev = load_data(IN_PATH + 'dev-v1.1-preprocessed.json')

## 2. Solutions

** Let's import evaluation module. **

In [3]:
import evaluation

** Let's define a generic function to run different methods/solutions/approaches. **

In [4]:
def run_method(method, question_words, candidate_answers_words):
    if method == 'random_solution':
        return random_solution(len(candidate_answers_words))
    elif 'kernel' in method:
        return kernel_solution(question_words, candidate_answers_words, method)
    elif 'sentence_similarity' in method:
        return sentence_similarity_solution(question_words, candidate_answers_words)
    
    return None


def run(dataset, method='random_solution'):
    results = {'Method': method, 'Prec@1': [], 'Prec@5': [], 'Prec@10': [],
               'AvgPrec': [], 'MAP': 0}
    idx = 0
    for article in dataset['data']:
        for qas_context in article['paragraphs']:
            # get the number of candidate answers
            nr_candidate_answers = qas_context['nr_candidate_answers']

            for qas in qas_context['qas']:
                # get answers' labels from context
                answer_labels = list()
                for answer in qas['answers']:
                    answer_labels.append(answer['answer_label'])
                    
                # trying to keep the same notation
                question_words = qas['question_lemmas_without_stopwords']
                candidate_answers_words = qas_context['context_sentences_lemmas_without_stopwords']
                y = answer_labels
                
                # run a method
                y_pred = run_method(method, question_words, candidate_answers_words)
                
                # evaluation
                results['Prec@1'].append(evaluation.precision_at_k(y_pred, y, k=1))
                #results['Prec@5'].append(evaluation.precision_at_k(y_pred, y, k=5))
                #results['Prec@10'].append(evaluation.precision_at_k(y_pred, y, k=10))
                results['AvgPrec'].append(evaluation.average_precision(y_pred, y))
                         
                
    # evaluation (MAP - mean average precision)
    results['MAP'] = np.mean(results['AvgPrec'])
    results['StdAP'] = np.std(results['AvgPrec'])
    results['AvgPrec@1'] = np.mean(results['Prec@1'])
    results['StdPrec@1'] = np.std(results['Prec@1'])
    #results['AvgPrec@5'] = np.mean(results['Prec@5'])
    #results['AvgPrec@10'] = np.mean(results['Prec@10'])
    
    return results

### 2.1 Random approach

In [5]:
import numpy as np

def random_solution(n):
    return np.random.permutation(n).tolist()

In [6]:
def write_results(results):
    print('Method: {}'.format(results['Method']))
    print('AvgPrec@1: {} (std = {})'.format(results['AvgPrec@1'], results['StdPrec@1']))
    print('MAP: {} (std = {})'.format(results['MAP'], results['StdAP']))
    print('\n')

In [7]:
train_results = run(train, method='random_solution')
write_results(train_results)

Method: random_solution
AvgPrec@1: 0.244215116611 (std = 0.429620871734)
MAP: 0.490218023302 (std = 0.311005750001)




### 2.2 String kernels

** I will try to use string kernels at word level even if they are used as character-level method. **

** Aux methods **

In [8]:
def num(word, sentence_words):
    count = 0
    for w in sentence_words:
        if word == w:
            count += 1
    
    return count

### 2.2.1 Spectrum kernel

In [9]:
def spectrum_kernel_value(question_words, sentence_words):
    kernel_value = 0
    vocab_inters = set(question_words).intersection(sentence_words)
    
    for word in vocab_inters:
        kernel_value += num(word, question_words) * num(word, sentence_words)
        
    return kernel_value

### 2.2.2 Presence kernel

In [10]:
def presence_kernel_value(question_words, sentence_words):
    kernel_value = 0
    vocab_inters = set(question_words).intersection(sentence_words)
    
    return len(vocab_inters)

### 2.2.3 Intersection kernel

In [11]:
def intersection_kernel_value(question_words, sentence_words):
    kernel_value = 0
    vocab_inters = set(question_words).intersection(sentence_words)
    
    for word in vocab_inters:
        kernel_value += min(num(word, question_words), num(word, sentence_words))
        
    return kernel_value

In [12]:
def kernel_solution(question_words, context_sentences, kernel_type='intersection_kernel'):
    scores = {}
    
    for sentence_label in range(len(context_sentences)):
        kernel_value = 0
        if kernel_type == 'spectrum_kernel':
            kernel_value = spectrum_kernel_value(question_words, context_sentences[sentence_label])
        elif kernel_type == 'presence_kernel':
            kernel_value = presence_kernel_value(question_words, context_sentences[sentence_label])
        elif kernel_type == 'intersection_kernel':
            kernel_value = intersection_kernel_value(question_words, context_sentences[sentence_label])
        
        scores[sentence_label] = kernel_value
        
    labels = sorted(scores, key=scores.get, reverse=True)
    
    return labels

In [13]:
methods = ['spectrum_kernel', 'presence_kernel', 'intersection_kernel']
for method in methods:
    dev_results = run(dev, method)
    write_results(dev_results)

Method: spectrum_kernel
AvgPrec@1: 0.779470198675 (std = 0.414603917073)
MAP: 0.866177981413 (std = 0.244744644951)


Method: presence_kernel
AvgPrec@1: 0.818070009461 (std = 0.38578681818)
MAP: 0.886540442806 (std = 0.230660234932)


Method: intersection_kernel
AvgPrec@1: 0.81816461684 (std = 0.385708797658)
MAP: 0.886673268564 (std = 0.230547762437)




### 2.3 Sentence similarity

** The code for max sentence similarity is taken from this site: https://nlpforhackers.io/wordnet-sentence-similarity/.  
This algorithm is proposed by Mihalcea et al. in the paper [Corpus-based and Knowledge-based Measures
of Text Semantic Similarity](https://www.aaai.org/Papers/AAAI/2006/AAAI06-123.pdf).  
More info about word similarities cand be found here: http://www.nltk.org/howto/wordnet.html. **

In [14]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 

def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
    
    return None

 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        similarities = [synset.wup_similarity(ss) for ss in synsets2]
        best_score = 0
        if similarities:
            best_score = max(similarities)
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    if count > 0:
        score /= count
    
    return score


def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 

In [15]:
def sentence_similarity_solution(question_words, candidate_answers_words):
    question_sentence = " ".join(question_words)
    scores = {}
    
    for sentence_label in range(len(candidate_answers_words)):
        candidate_answer_sentence = " ".join(candidate_answers_words[sentence_label])
        similarity = symmetric_sentence_similarity(question_sentence, candidate_answer_sentence)
        scores[sentence_label] = similarity
        
    labels = sorted(scores, key=scores.get, reverse=True)
    
    return labels

## 3. Results

** First results **

In [16]:
methods = ['random_solution', 'spectrum_kernel', 'presence_kernel', 'intersection_kernel']#, 'sentence_similarity']

print('---Results---train_set---')
for method in methods:
    train_results = run(train, method)
    write_results(train_results)
    
print('---Results---dev_set---')
for method in methods:
    dev_results = run(dev, method)
    write_results(dev_results)


---Results---train_set---
Method: random_solution
AvgPrec@1: 0.244489092341 (std = 0.429783871344)
MAP: 0.490277282186 (std = 0.311068696302)


Method: spectrum_kernel
AvgPrec@1: 0.757873948333 (std = 0.428370198276)
MAP: 0.85650946151 (std = 0.260469574575)


Method: presence_kernel
AvgPrec@1: 0.785305768331 (std = 0.410610056571)
MAP: 0.871736089368 (std = 0.251560020315)


Method: intersection_kernel
AvgPrec@1: 0.785477003162 (std = 0.410491023855)
MAP: 0.8718333127 (std = 0.251494647526)


---Results---dev_set---
Method: random_solution
AvgPrec@1: 0.263008514664 (std = 0.440267005212)
MAP: 0.502599233445 (std = 0.308659968528)


Method: spectrum_kernel
AvgPrec@1: 0.779470198675 (std = 0.414603917073)
MAP: 0.866177981413 (std = 0.244744644951)


Method: presence_kernel
AvgPrec@1: 0.818070009461 (std = 0.38578681818)
MAP: 0.886540442806 (std = 0.230660234932)


Method: intersection_kernel
AvgPrec@1: 0.81816461684 (std = 0.385708797658)
MAP: 0.886673268564 (std = 0.230547762437)


